# Demand Forecast
   
---
   
This project intended to predict the demand that define by streaming time in time-series manner. The data used for this project will aggregate several datasets that been request from Spotify of my Personal streaming history. 
     
---
  

# 1. Streaming

## Data Overview

---    
   
The datasets used is `Streaming_History_Audio_#.json` which provided detailed streaming history and has long time range that adequate for training a model. The data that will be used in this project consists of:   
   
*  `ts`: This field is a timestamp indicating when the track stopped playing in UTC (Coordinated Universal Time). The order is year, month and day followed by a timestamp in military time.   
    
*  `ms_played`: This field is the number of milliseconds the stream was played.   
   
*  `platform`: This field is the platform used when streaming the track (e.g. Android OS, Google Chromecast).    
   
*  `conn_country`: This field is the country code of the country where the stream was played (e.g. SE - Sweden).   
   
*  `reason_start`: This field is a value telling why the track started (e.g. “trackdone”)   
   
*  `reason_end`: This field is a value telling why the track ended (e.g. “endplay”).   
   
*  `shuffle`: This field has the value True or False depending on if shuffle mode was used when playing the track.   
   
*  `skipped`: This field indicates if the user skipped to the next song.   
   
*  `offline`: This field indicates whether the track was played in offline mode (“True”) or not (“False”).    
   
*  `incognito_mode`: This field indicates whether the track was played during a private session (“True”) or not (“False”).   
   
---

## Library

In [1]:
# Import Library
import os

## Universal Data Processing
import numpy as np
import pandas as pd

## Regular Expression for Text Data
import re

## JSON Files Manipulation
import json

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

*Display Settings*

In [11]:
pd.set_option("display.max_columns", None)   # show all columns
pd.set_option("display.width", None)         # auto-detect width
pd.set_option("display.max_colwidth", None)  # don't truncate cell content

## Data Preparation

*Load and Merge Data*

In [14]:
# Get Current Directory Address
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, "Dataset")

# List included datasets paths
paths = [
    os.path.join(dataset_dir, "Streaming_History_Audio_2018-2019_0.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2019-2020_1.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2020_2.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2020-2021_3.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2021-2022_4.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2022-2023_5.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2023_6.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2023-2024_7.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2024-2025_8.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2025_9.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2025-2026_10.json"),
    os.path.join(dataset_dir, "Streaming_History_Audio_2026_11.json"),
]

# Load each datasets
all_data = []

for idx, path in enumerate(paths):              # use `for count, item in enumerate(items, start=1)` to customize the indexing
    print(f"Loading file {idx}: {path}")
    with open(path, "r", encoding="utf-8") as json_file:
        data_idx = json.load(json_file)
        all_data.append(data_idx)

print("Files loaded", len(all_data))

Loading file 0: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2018-2019_0.json
Loading file 1: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2019-2020_1.json
Loading file 2: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2020_2.json
Loading file 3: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2020-2021_3.json
Loading file 4: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2021-2022_4.json
Loading file 5: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2022-2023_5.json
Loading file 6: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2023_6.json
Loading file 7: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2023-2024_7.json
Loading file 8: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2024-2025_8.json
Loading file 9: c:\03. Other\Spotify_Unwrapped\Dataset\Streaming_History_Audio_2025_9.json
Loading file 10: c:\03. Other\Spotify_Unwrapped\Dataset

*Flatten JSON data to Dataframe*

In [ ]:
# Convert loaded data into suitable objects for ease of manipulation
flat_data = []

for file_data in all_data:
    flat_data.extend(file_data)

# Flatten the Table to make sure the dictionary or embedded arays data in the flat table
df_json = pd.json_normalize(flat_data)

df_json.head()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2018-02-05T00:58:40Z,"Android OS 5.1.1 API 22 (OPPO, F1f)",9887,ID,A Different Way (with Lauv),DJ Snake,A Different Way (with Lauv),spotify:track:0Wv5wuenRLI3BcwgT3HPIP,NaN,NaN,NaN,None,None,None,None,playbtn,fwdbtn,True,False,False,NaN,False
1,2018-02-05T00:59:02Z,"Android OS 5.1.1 API 22 (OPPO, F1f)",11193,ID,The Story Never Ends,Lauv,The Story Never Ends,spotify:track:5cDNs4utoHt0WNRZuziews,NaN,NaN,NaN,None,None,None,None,fwdbtn,fwdbtn,True,False,False,NaN,False
2,2018-02-05T00:59:26Z,"Android OS 5.1.1 API 22 (OPPO, F1f)",11463,ID,Don't Matter - Recorded at Spotify Studios NYC,Lauv,Spotify Singles,spotify:track:4a00SVbdG6saNqRJlC4XKQ,NaN,NaN,NaN,None,None,None,None,fwdbtn,fwdbtn,True,False,False,NaN,False
3,2018-02-05T00:59:48Z,"Android OS 5.1.1 API 22 (OPPO, F1f)",7826,ID,Comfortable,Lauv,Lost in the Light,spotify:track:4DPezINQNrkmmzjasvltzW,NaN,NaN,NaN,None,None,None,None,fwdbtn,endplay,True,False,False,NaN,False
4,2018-02-05T01:03:51Z,"Android OS 5.1.1 API 22 (OPPO, F1f)",236759,ID,Falling,Fancy Feelings,Falling,spotify:track:5HeufMDrbfDWQj7AZlhlky,NaN,NaN,NaN,None,None,None,None,playbtn,trackdone,True,False,False,NaN,False


In [21]:
df_json.columns

Index(['ts', 'platform', 'ms_played', 'conn_country',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='str')

*Selected Columns*

In [16]:
selected_columns = [
    "ts",
    "ms_played",
    "platform",
    "conn_country",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped",
    "offline",
    "incognito_mode",
]

df_demand = df_json[selected_columns].copy()

df_demand.head()

,ts,ms_played,platform,conn_country,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
0,2018-02-05T00:58:40Z,9887,"Android OS 5.1.1 API 22 (OPPO, F1f)",ID,playbtn,fwdbtn,True,False,False,False
1,2018-02-05T00:59:02Z,11193,"Android OS 5.1.1 API 22 (OPPO, F1f)",ID,fwdbtn,fwdbtn,True,False,False,False
2,2018-02-05T00:59:26Z,11463,"Android OS 5.1.1 API 22 (OPPO, F1f)",ID,fwdbtn,fwdbtn,True,False,False,False
3,2018-02-05T00:59:48Z,7826,"Android OS 5.1.1 API 22 (OPPO, F1f)",ID,fwdbtn,endplay,True,False,False,False
4,2018-02-05T01:03:51Z,236759,"Android OS 5.1.1 API 22 (OPPO, F1f)",ID,playbtn,trackdone,True,False,False,False


*Consideration*   
   
*  `master_metadata_track_name`, `master_metadata_album_artist_name`,`master_metadata_album_album_name`, `spotify_track_uri` : There are none metadata/track detailed database that can be access so this columns become unuseable.   
    
*  `episode_name`, `episode_show_name`, `spotify_episode_uri`, `audiobook_title`,`audiobook_uri`, `audiobook_chapter_uri`, `audiobook_chapter_title` : NULL

## Data Preprocessing

In [22]:
df_demand.info()

<class 'pandas.DataFrame'>
RangeIndex: 175923 entries, 0 to 175922
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   ts              175923 non-null  str  
 1   ms_played       175923 non-null  int64
 2   platform        175923 non-null  str  
 3   conn_country    175923 non-null  str  
 4   reason_start    175923 non-null  str  
 5   reason_end      175923 non-null  str  
 6   shuffle         175923 non-null  bool 
 7   skipped         175923 non-null  bool 
 8   offline         175923 non-null  bool 
 9   incognito_mode  175923 non-null  bool 
dtypes: bool(4), int64(1), str(5)
memory usage: 8.7 MB


*  `ts`: Need to convert to datatime datatype then extract each of date and time-related data for training (e.g. `Day`, `Hour-Decimal`).   
   
*  `ms_played`: Convert to minute, hour.
   
*  `platform`, `conn_country`, `reason_start`, `reason_end`: Categorical data with fixed categorization (assumption), easier to be **Label Encoding**.   
   
*  `shuffle`, `skipped`, `offline`, `incognito_mode`: Bool data will be **One-Hot / Binary Encoding** to be able train in Regression Model.

*Encode Categorical data*

In [26]:
# pip install scikit-learn

In [25]:
# Library for Data Preprocessing
from sklearn.preprocessing import LabelEncoder

In [33]:
# Label Encoding
le = LabelEncoder()

df_demand["platform_encoded"] = le.fit_transform(df_demand["platform"])
df_demand["conn_country_encoded"] = le.fit_transform(df_demand["conn_country"])
df_demand["reason_start_encoded"] = le.fit_transform(df_demand["reason_start"])
df_demand["reason_end_encoded"] = le.fit_transform(df_demand["reason_end"])

# Binary Encoding
df_demand["shuffle_binary"] = df_demand["shuffle"].astype(int)
df_demand["skipped_binary"] = df_demand["skipped"].astype(int)
df_demand["offline_binary"] = df_demand["offline"].astype(int)
df_demand["incognito_mode_binary"] = df_demand["incognito_mode"].astype(int)

# Filter out the encoded columns for modeling
exlude_columns = [
    "platform",
    "conn_country",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped",
    "offline",
    "incognito_mode",
]

df_encoded = df_demand.drop(columns=exlude_columns)

df_encoded.head()

,ts,ms_played,platform_encoded,conn_country_encoded,reason_start_encoded,reason_end_encoded,shuffle_binary,skipped_binary,offline_binary,incognito_mode_binary
0,2018-02-05T00:58:40Z,9887,4,4,4,2,1,0,0,0
1,2018-02-05T00:59:02Z,11193,4,4,3,2,1,0,0,0
2,2018-02-05T00:59:26Z,11463,4,4,3,2,1,0,0,0
3,2018-02-05T00:59:48Z,7826,4,4,3,1,1,0,0,0
4,2018-02-05T01:03:51Z,236759,4,4,4,6,1,0,0,0


*Feature Engingeering*

In [34]:
# Temporal Feature Engineering
# Extracting Time-Based Features
df_encoded["ts"] = pd.to_datetime(df_encoded["ts"])

# Date-Based Features
df_encoded["Day"]               = df_encoded["ts"].dt.day
df_encoded["Month"]             = df_encoded["ts"].dt.month
df_encoded["Year"]              = df_encoded["ts"].dt.year
df_encoded["day_of_week"]       = df_encoded["ts"].dt.dayofweek + 1
df_encoded["week_of_year"]      = df_encoded["ts"].dt.isocalendar().week

# Time-Based Features
df_encoded["Second"]            = df_encoded["ts"].dt.second
df_encoded["Minute"]            = df_encoded["ts"].dt.minute
df_encoded["Hour"]              = df_encoded["ts"].dt.hour

# Feature Generation
# Date Combination Features
df_encoded['Month-Year']        = df_encoded['Month'].astype(str) + '-' + df_encoded['Year'].astype(str)
df_encoded['Day-Month']         = df_encoded['Day'].astype(str) + '-' + df_encoded['Month'].astype(str)
df_encoded['Hour-Day']          = df_encoded['Hour'].astype(str) + '-' + df_encoded['Day'].astype(str)

# Drop the original timestamp
df_encoded.drop(columns=["ts"], inplace=True)

df_encoded.head()

,ms_played,platform_encoded,conn_country_encoded,reason_start_encoded,reason_end_encoded,shuffle_binary,skipped_binary,offline_binary,incognito_mode_binary,Day,Month,Year,day_of_week,week_of_year,Second,Minute,Hour,Month-Year,Day-Month,Hour-Day
0,9887,4,4,4,2,1,0,0,0,5,2,2018,1,6,40,58,0,2-2018,5-2,0-5
1,11193,4,4,3,2,1,0,0,0,5,2,2018,1,6,2,59,0,2-2018,5-2,0-5
2,11463,4,4,3,2,1,0,0,0,5,2,2018,1,6,26,59,0,2-2018,5-2,0-5
3,7826,4,4,3,1,1,0,0,0,5,2,2018,1,6,48,59,0,2-2018,5-2,0-5
4,236759,4,4,4,6,1,0,0,0,5,2,2018,1,6,51,3,1,2-2018,5-2,1-5


In [31]:
df_encoded.info()

<class 'pandas.DataFrame'>
RangeIndex: 175923 entries, 0 to 175922
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ms_played              175923 non-null  int64 
 1   platform_encoded       175923 non-null  int64 
 2   conn_country_encoded   175923 non-null  int64 
 3   reason_start_encoded   175923 non-null  int64 
 4   reason_end_encoded     175923 non-null  int64 
 5   shuffle_binary         175923 non-null  int64 
 6   skipped_binary         175923 non-null  int64 
 7   offline_binary         175923 non-null  int64 
 8   incognito_mode_binary  175923 non-null  int64 
 9   Day                    175923 non-null  int32 
 10  Month                  175923 non-null  int32 
 11  Year                   175923 non-null  int32 
 12  day_of_week            175923 non-null  int32 
 13  week_of_year           175923 non-null  UInt32
 14  Month-Year             175923 non-null  str   
 15  Day-Month  

*  `Month-Year`, `Day-Month`, `Hour-Day` need to be Encode for Regression Training.

In [35]:
# Label Encoding
df_encoded["Month-Year_encoded"] = le.fit_transform(df_encoded["Month-Year"])
df_encoded["Day-Month_encoded"] = le.fit_transform(df_encoded["Day-Month"])
df_encoded["Hour-Day_encoded"] = le.fit_transform(df_encoded["Hour-Day"])

# Drop the original combined features
df_encoded.drop(columns=["Month-Year", "Day-Month", "Hour-Day"], inplace=True)

df_encoded.head()

,ms_played,platform_encoded,conn_country_encoded,reason_start_encoded,reason_end_encoded,shuffle_binary,skipped_binary,offline_binary,incognito_mode_binary,Day,Month,Year,day_of_week,week_of_year,Second,Minute,Hour,Month-Year_encoded,Day-Month_encoded,Hour-Day_encoded
0,9887,4,4,4,2,1,0,0,0,5,2,2018,1,6,40,58,0,29,310,26
1,11193,4,4,3,2,1,0,0,0,5,2,2018,1,6,2,59,0,29,310,26
2,11463,4,4,3,2,1,0,0,0,5,2,2018,1,6,26,59,0,29,310,26
3,7826,4,4,3,1,1,0,0,0,5,2,2018,1,6,48,59,0,29,310,26
4,236759,4,4,4,6,1,0,0,0,5,2,2018,1,6,51,3,1,29,310,57


## Data Transformation

**Functions**

*References Code*

**Data Exploration**

***Findings***   
   
*  
    
    
***Transformation Plan***   
   
*   

## Model Training

In [36]:
# Import libraries
import numpy as np
import math

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge, Lasso, LinearRegression, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer

*References Codes*

In [ ]:
# Split into features (X) and target (y)
X_reg = train.drop(columns=["outperform_binary", "excess_return"], errors="ignore")
y_reg = train["excess_return"]

# Drop Missing Values
X_reg = X_reg.dropna()
y_reg = y_reg.loc[X_reg.index]

# Recompute feature groups from X only
categorical_features = X_reg.select_dtypes(include=["object", "string"]).columns.tolist()
numerical_features   = X_reg.select_dtypes(include=[np.number]).columns.tolist()

# Separate numerical groups
return_volatility_features = [c for c in numerical_features if "return" in c or "volatility" in c]
volume_features = [c for c in numerical_features if "volume" in c]
remaining_numerical = [c for c in numerical_features if c not in volume_features + return_volatility_features]

In [ ]:
# Data Pre-Preprocessor for Regression
preprocessor_reg = ColumnTransformer(
    transformers=[
        # Returns & volatility
        ("returns_vol", Pipeline([
            ('imputer', SimpleImputer(strategy="mean")),
            ("robust", RobustScaler())
        ]), return_volatility_features),

        # Volumes
        ("volumes", Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
            ("log", FunctionTransformer(np.log1p, validate=False)),
            ("scaler", StandardScaler())
        ]), volume_features),

        # Categorical
        ("categorical", Pipeline([
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features),

        # Remaining numeric
        ("remaining", Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), remaining_numerical),
    ],
    remainder="drop"
)

In [ ]:
# Models
models_reg = {
    "Ridge": (Ridge(random_state=42),
              {"clf__alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}),
    "Lasso": (Lasso(random_state=42, max_iter=100000),
              {"clf__alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}),
    "SVR": (SVR(),
            {"clf__kernel": ["rbf"],
             "clf__C": [0.001, 0.01, 0.1, 1.0, 10.0],
             "clf__gamma": ["scale"]})
}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

best_model = None
best_params = None
best_rmse = np.inf

print("\n5-Fold Cross-Validation Results (RMSE)")
for name, (model, param_grid) in models_reg.items():
    clf = Pipeline([
        ("preprocessor", preprocessor_reg),
        ("clf", model)
    ])

    grid = GridSearchCV(
        estimator=clf,
        param_grid=param_grid,
        cv=tscv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )
    grid.fit(X_reg, y_reg)

    mean_rmse = -grid.best_score_
    print(f"{name:<15} → Best RMSE = {mean_rmse:.4f} | Params: {grid.best_params_}")

    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_model = grid.best_estimator_
        best_params = grid.best_params_

print("\n=== Best Overall Model ===")
print(best_model)
print("Params:", best_params)
print("Best CV RMSE:", best_rmse)